In [17]:
import pandas as pd
from zmq.backend import first

labels_list = [
            '11','12','13','14','15','16','17',
            '21','22','23','24','25','26','27',
            '31','32','33','34','35','36','37',
            '41','42','43','44','45','46','47',
            '51','52','53','54','55',
            '61','62','63','64','65',
            '71','72','73','74','75',
            '81','82','83','84','85'
            ]
df = pd.read_pickle('raw_newdata.pkl')
print(f"检测图片总数：{len(df)}")
print(f"标签总数:{len(labels_list)}")
labels_size = len(labels_list)
less_then = 0
more_then = 0
equals = 0
total_count = 0
print("-------------------------------------")
for item in df:
    prd_instance = item['pred_instances']
    total_count += prd_instance['labels'].shape[0]
    print(prd_instance['labels'])
    if labels_size == prd_instance['labels'].shape[0]:
        equals+=1
    elif prd_instance['labels'].shape[0] > labels_size:
        more_then+=1
    else:
        less_then+=1

print("-------------------------------------")
print(f"检测图片总数：{len(df)}其中,与标签数相等的有:{equals},小于的：{more_then},大于的:{less_then},共检出牙齿:{total_count}")

检测图片总数：121
标签总数:48
-------------------------------------
tensor([21, 22,  7, 45,  0,  1, 15, 30, 28, 44, 29, 35, 41, 11, 14,  9, 10, 19,
        39, 12, 13, 34, 46, 27, 26, 40, 43,  4, 36,  5,  3, 16, 37, 38,  8, 23,
        20, 31,  2, 47,  6, 24, 17, 42, 33, 32, 32,  5,  7, 18,  1,  4, 39, 19,
        25])
tensor([12, 26, 41, 19,  0, 17, 14, 30,  8,  7, 13,  9, 42,  5, 21, 23,  6, 22,
        20, 16, 11, 15, 27, 18, 45, 37, 24, 25, 10,  1, 32,  2, 40, 31, 47, 46,
         4, 36,  3, 29,  3, 35,  3, 36, 31, 46,  0,  2, 30, 10, 34, 12])
tensor([31,  6, 36,  0, 42,  1, 29,  9,  2, 46,  8, 30,  7, 35, 27, 37, 16, 26,
         3, 10, 19, 23,  5, 32,  4, 12, 41, 25, 20, 18, 11, 13, 45, 47, 33, 17,
        24, 40, 28, 14, 21, 34, 15, 38, 43, 22, 34, 44, 39, 33, 45])
tensor([14, 21,  9, 18, 15, 19,  3, 26, 30, 12, 23, 16, 37, 31, 29,  8, 42, 20,
        22,  7,  5, 45,  2, 32,  1, 11,  4, 10, 27, 40, 41, 13, 46, 24, 25, 35,
        17,  6, 47, 36, 34,  0, 29, 39, 37, 41, 28, 39, 35])
tensor(

当前牙齿的评估标准为：
灵敏度、精确度、F-1 分数和平均精确度-0.5 (mAP-0.5)


在COCO的评估中，IoU是一个关键的阈值。一个预测结果是否被认为是真正的正样本（True Positive, TP），取决于其与真实框的IoU是否达到某个预设的阈值。
AP的计算步骤：
确定IoU阈值：COCO的主要AP指标是在多个IoU阈值下计算的，从0.5到0.95，步长为0.05（即0.5, 0.55, 0.6, ..., 0.95）。

生成精确率-召回率曲线 (Precision-Recall Curve)：

对于每个类别，模型会输出一系列的检测框，并附带一个置信度分数（confidence score）。

通过不断改变置信度阈值，可以得到不同的精确率（Precision）和召回率（Recall）组合。

精确率：在所有被模型预测为正样本的结果中，真正是正样本的比例。

召回率：在所有真实的正样本中，被模型成功预测出来的比例。

以召回率为横轴，精确率为纵轴，就可以绘制出该类别的P-R曲线。

计算AP：AP是P-R曲线下的面积。COCO采用101个召回率点（0, 0.01, ..., 1.0）进行插值来计算曲线下面积，从而得到在该IoU阈值下的AP值。

求取平均AP：将步骤1中10个IoU阈值下计算出的AP值进行平均，就得到了COCO最主要的评价指标，通常表示为 AP 或 AP@[.50:.05:.95]。

In [6]:
from pycocotools.coco import COCO
import json

annFile = r'data/dataset/coco/crop_child/annotations/test.json'

coco = COCO(annFile)
# 获取 COCO 数据集中的所有类别列表
cats = coco.loadCats(coco.getCatIds())
cat_names = [cat['name'] for cat in cats]
print(f"COCO 数据集共有 {len(cat_names)} 个类别: {cat_names}")


# 创建一个字典来存储每个类别的统计数量
category_stats = {}

# 遍历所有类别
for cat in cats:
    cat_id = cat['id']
    cat_name = cat['name']

    # 获取当前类别所有的标注图像ID
    img_ids = coco.getImgIds(catIds=[cat_id])

    # 获取当前类别所有的标注实例（注意：一张图片中可以有多个同类别的实例）
    ann_ids = coco.getAnnIds(catIds=[cat_id])
    annotations = coco.loadAnns(ann_ids)

    # 统计实例数量
    instance_count = len(annotations)

    # 将结果存入字典
    category_stats[cat_name] = {
        'category_id': cat_id,
        'instance_count': instance_count,
        'image_count': len(img_ids) # 顺带统计包含此类的图片数量
    }

# 按实例数量从多到少排序
sorted_stats = dict(sorted(category_stats.items(), key=lambda item: item[1]['instance_count'], reverse=True))

# 打印结果
print("\n===== 类别实例数量统计 (从多到少) =====")
t_count = 0

for cat_name, info in sorted_stats.items():
    t_count += info['instance_count']
    print(f"{cat_name} (ID: {info['category_id']}): {info['instance_count']} 个实例, 出现在 {info['image_count']} 张图片中")

print(f"总标签数：{t_count}")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
COCO 数据集共有 48 个类别: ['11', '12', '13', '14', '15', '16', '17', '21', '22', '23', '24', '25', '26', '27', '31', '32', '33', '34', '35', '36', '37', '41', '42', '43', '44', '45', '46', '47', '51', '52', '53', '54', '55', '61', '62', '63', '64', '65', '71', '72', '73', '74', '75', '81', '82', '83', '84', '85']

===== 类别实例数量统计 (从多到少) =====
11 (ID: 1): 121 个实例, 出现在 121 张图片中
16 (ID: 6): 121 个实例, 出现在 121 张图片中
13 (ID: 3): 120 个实例, 出现在 120 张图片中
14 (ID: 4): 120 个实例, 出现在 120 张图片中
21 (ID: 9): 120 个实例, 出现在 120 张图片中
23 (ID: 11): 120 个实例, 出现在 120 张图片中
24 (ID: 12): 120 个实例, 出现在 120 张图片中
34 (ID: 20): 120 个实例, 出现在 120 张图片中
36 (ID: 22): 120 个实例, 出现在 120 张图片中
44 (ID: 28): 120 个实例, 出现在 120 张图片中
46 (ID: 30): 120 个实例, 出现在 120 张图片中
26 (ID: 14): 119 个实例, 出现在 119 张图片中
33 (ID: 19): 119 个实例, 出现在 119 张图片中
37 (ID: 23): 119 个实例, 出现在 119 张图片中
47 (ID: 31): 119 个实例, 出现在 119 张图片中
12 (ID: 2): 118 个实例, 出现在 118 张图片中
15 (ID: 5): 118 个实例, 出现在 1

总标签数：5012

检出标签数：6122

In [16]:
import numpy as np
IOU_THRESHOLD = 0.5 # IOU 阈值
SCORE_THRESHOLD = 0.3# 置信度阈值

def calculate_iou(boxA,boxB):
    """
    计算两个边界框的IOU
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0,xB - xA) * max(0,yB - yA)

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

per_image_stats = {}
per_class_stats = {
    cls_id:{'TP':0,'FP':0,'FN':0,'iou_sum':0,'tp_count':0} for cls_id in coco.getCatIds()
}

img_ids_in_gt = sorted(coco.getImgIds())
print(img_ids_in_gt)

print(f'\n开始分析{len(img_ids_in_gt)}张图片')

for i,gt_img_id in enumerate(img_ids_in_gt):
    # 初始化当前图片的统计
    per_image_stats[gt_img_id] = {'FP': 0, 'FN': 0, 'TP': 0, 'filename': coco.loadImgs(gt_img_id)[0]['file_name']}
    prd_result = df[i]
    assert prd_result['img_id'] == gt_img_id , f'图片ID不相符，GT={img_id},Pred={prd_result['img_id']}'

    pred_instance = prd_result['pred_instances']

    #置信度阈值
    score_mask = pred_instance['scores'] >= SCORE_THRESHOLD
    pred_bboxes = pred_instance['bboxes'][score_mask].cpu().numpy()
    pred_labels = pred_instance['labels'][score_mask].cpu().numpy()
    pred_scores = pred_instance['scores'][score_mask].cpu().numpy()

    # 获取当前图片的真实标注
    ann_ids = coco.getAnnIds(imgIds=img_id)
    gt_anns = coco.loadAnns(ann_ids)
    gt_bboxes = np.array([ann['bbox'] for ann in gt_anns])
    gt_labels = np.array([ann['category_id'] for ann in gt_anns])

    #  将 [x, y, w, h] 转换为 [x1, y1, x2, y2]
    if gt_bboxes.shape[0] > 0:
        gt_bboxes[:, 2] += gt_bboxes[:, 0]
        gt_bboxes[:, 3] += gt_bboxes[:, 1]

    # 记录每个GT框是否已被匹配
    gt_matched = np.zeros(gt_bboxes.shape[0], dtype=bool)

    # FP计数：先假设所有预测都是错的
    num_fp_on_img = len(pred_bboxes)
    num_tp_on_img = 0




[14, 24, 52, 55, 62, 66, 89, 94, 97, 115, 134, 140, 147, 151, 166, 168, 183, 191, 194, 196, 204, 214, 217, 233, 237, 244, 268, 296, 298, 305, 326, 331, 336, 339, 340, 346, 347, 363, 406, 420, 442, 445, 448, 453, 460, 462, 464, 470, 479, 482, 489, 514, 518, 527, 551, 554, 555, 559, 561, 565, 576, 582, 608, 617, 618, 663, 665, 686, 713, 724, 737, 763, 772, 779, 780, 786, 797, 814, 816, 817, 821, 828, 853, 860, 864, 906, 912, 913, 926, 942, 977, 986, 997, 998, 1005, 1019, 1076, 1088, 1100, 1111, 1154, 1164, 1169, 1174, 1188, 1202, 1203, 1211, 1224, 1243, 1250, 1252, 1257, 1266, 1270, 1272, 1275, 1276, 1277, 1279, 1282]

开始分析121张图片
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
11